In [121]:
from google import genai
from google.genai import types
import os

In [122]:
MODEL = 'gemini-2.5-flash'

In [123]:
client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

In [124]:
import json

# config = types.GenerateContentConfig(
#     system_instruction="""Provide a short answer to the query. Format your response as JSON:
# {
#   "answer": "your answer here",
#   "search_query": "a search query to support your key claim"
# }"""
# )

config = types.GenerateContentConfig(
    system_instruction="""Provide a SHORT answer to the query. Your answer should be clear, concise and get to the point."""
)

In [125]:
import requests
import html
from bs4 import BeautifulSoup

def fetch_webpage_content(url: str) -> str:
    """Fetch and extract text content from a webpage"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
        
        soup = BeautifulSoup(resp.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        
        # Get text
        text = soup.get_text()
        
        # Clean up whitespace
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text
    except Exception as e:
        return f"Error fetching webpage: {str(e)}"

def search_brave_and_extract_quote(query: str, argument: str) -> str:
    """
    Two-step process:
    1. Search using Brave to find relevant webpage
    2. Fetch webpage and use LLM to extract direct quote supporting the argument
    """
    api_key = os.environ.get('BRAVE_SEARCH_API_KEY')
    if not api_key:
        return "BRAVE_SEARCH_API_KEY not set"
    
    try:
        # Step 1: Brave search
        url = "https://api.search.brave.com/res/v1/web/search"
        headers = {
            'Accept': 'application/json',
            'X-Subscription-Token': api_key
        }
        # params = {'q': f"{query} site:wikipedia.org", 'count': 1}
        params = {'q': query, 'count': 1}
        
        resp = requests.get(url, headers=headers, params=params)
        resp.raise_for_status()
        data = resp.json()
        
        results = data.get('web', {}).get('results', [])
        if not results:
            return "No results found"
        
        # Get the top result
        item = results[0]
        page_url = item.get('url', '')
        title = item.get('title', '')
        
        if not page_url:
            return "No valid URL found"
        
        # Step 2: Fetch webpage content
        webpage_content = fetch_webpage_content(page_url)
        
        if webpage_content.startswith("Error"):
            return webpage_content
        
        # Step 3: Extract direct quote using LLM
        quote = extract_direct_quote(webpage_content, argument, page_url)
        
        return quote
        
    except Exception as e:
        return f"Error: {str(e)}"

In [126]:
def extract_direct_quote(webpage_content: str, argument: str, source_url: str) -> str:
    """Use LLM to extract the best direct quote from webpage that supports the argument"""
    
    # Truncate content if too long (keep first 10000 chars to stay within token limits)
    if len(webpage_content) > 10000:
        webpage_content = webpage_content[:10000] + "..."
    
    prompt = f"""Given the following webpage content and a debater's argument, extract the BEST direct quote from the webpage that supports the argument.

CRITICAL REQUIREMENTS:
1. The quote MUST be a direct, verbatim excerpt from the webpage (no paraphrasing)
2. Maximum 3 consecutive sentences
3. The sentences must be contiguous (adjacent to each other)
4. Choose the quote that most directly supports the argument

Webpage content:
{webpage_content}

Debater's argument:
{argument}

Return ONLY the direct quote as a JSON object:
{{
  "quote": "the exact verbatim quote from the webpage"
}}

If no suitable quote is found, return:
{{
  "quote": "No suitable quote found"
}}"""

    try:
        response = client.models.generate_content(
            model=MODEL,
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=0.0  # Use deterministic extraction
            )
        )
        
        text = response.text.strip()
        
        # Extract JSON
        json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', text, re.DOTALL)
        if json_match:
            text = json_match.group(1)
        else:
            json_match = re.search(r'\{.*\}', text, re.DOTALL)
            if json_match:
                text = json_match.group(0)
        
        data = json.loads(text)
        quote = data.get('quote', 'Error: No quote in response')
        
        # Format with source
        return f'"{quote}" [Source: {source_url}]'
        
    except Exception as e:
        return f"Error extracting quote: {str(e)}"

In [145]:
import re

def format_question_for_display(text):
    """Format question with better rendering of mathematical notation"""
    
    # Replace common LaTeX patterns with Unicode/readable equivalents
    replacements = [
        # Greek letters
        (r'\\alpha', 'α'),
        (r'\\beta', 'β'),
        (r'\\gamma', 'γ'),
        (r'\\delta', 'δ'),
        (r'\\sigma', 'σ'),
        (r'\\theta', 'θ'),
        (r'\\lambda', 'λ'),
        (r'\\mu', 'μ'),
        (r'\\pi', 'π'),
        (r'\\omega', 'ω'),
        
        # Quantum mechanics notation
        (r'\|\\uparrow\\rangle', '|↑⟩'),
        (r'\|\\downarrow\\rangle', '|↓⟩'),
        (r'\\langle', '⟨'),
        (r'\\rangle', '⟩'),
        
        # Math operators
        (r'\\times', '×'),
        (r'\\cdot', '·'),
        (r'\\pm', '±'),
        (r'\\leq', '≤'),
        (r'\\geq', '≥'),
        (r'\\neq', '≠'),
        (r'\\approx', '≈'),
        (r'\\sqrt', '√'),
        
        # Subscripts (simple cases)
        (r'\\sigma\{z\}', 'σ_z'),
        (r'\\sigma\{x\}', 'σ_x'),
        (r'\\sigma\{y\}', 'σ_y'),
        (r'\\sigma_\{([a-z])\}', r'σ_\1'),
        (r'_\{([^}]+)\}', r'_\1'),
        
        # Clean up remaining backslashes and braces for simple cases
        (r'\{', ''),
        (r'\}', ''),
    ]
    
    result = text
    for pattern, replacement in replacements:
        result = re.sub(pattern, replacement, result)
    
    return result

# Test the formatter
test_text = r"A spin-half particle is in a linear superposition 0.5|\uparrow\rangle+sqrt(3)/2|\downarrow\rangle of its spin-up and spin-down states. If |\uparrow\rangle and |\downarrow\rangle are the eigenstates of \sigma{z} , then what is the expectation value up to one decimal place, of the operator 10\sigma{z}+5\sigma_{x} ?"
print("Before:")
print(test_text)
print("\nAfter:")
print(format_question_for_display(test_text))

Before:
A spin-half particle is in a linear superposition 0.5|\uparrow\rangle+sqrt(3)/2|\downarrow\rangle of its spin-up and spin-down states. If |\uparrow\rangle and |\downarrow\rangle are the eigenstates of \sigma{z} , then what is the expectation value up to one decimal place, of the operator 10\sigma{z}+5\sigma_{x} ?

After:
A spin-half particle is in a linear superposition 0.5|↑⟩+sqrt(3)/2|↓⟩ of its spin-up and spin-down states. If |↑⟩ and |↓⟩ are the eigenstates of σz , then what is the expectation value up to one decimal place, of the operator 10σz+5σ_x ?


In [146]:
from datasets import load_dataset
import random

# CONFIGURATION: Choose dataset
DATASET_TYPE = "GPQA"  # Options: "MMLU" or "GPQA"

if DATASET_TYPE == "MMLU":
    # Load MMLU dataset (fixed version)
    dataset = load_dataset("edinburgh-dawg/mmlu-redux-2.0", 'professional_law', split="test")
    dataset = [q for q in dataset if q['error_type'] == 'ok']
    for q in dataset:
        q['question'] = "Answer assuming US jurisdiction and practice: " + q['question']
    
    # Select a random question
    random_idx = random.randint(0, len(dataset) - 1)
    # random_idx = 28  # Hardcode for reproducibility
    
    question_data = dataset[random_idx]
    question = question_data['question']
    choices = question_data['choices']
    correct_idx = question_data['answer']

elif DATASET_TYPE == "GPQA":
    # Load GPQA dataset
    # Try the formatted version first, fall back to original if needed
    try:
        dataset = load_dataset("Idavidrein/gpqa", "gpqa_main", split="train")
    except:
        print("Note: If dataset fails to load, you may need to accept terms at https://huggingface.co/datasets/Idavidrein/gpqa")
        raise
    
    # Select a random question
    random_idx = random.randint(0, len(dataset) - 1)
    random_idx = 7  # Hardcode for reproducibility
    
    question_data = dataset[random_idx]
    
    # GPQA uses different field names - adjust based on actual structure
    # Common possibilities: 'Question'/'question', 'Correct Answer'/'correct_answer'
    if 'Question' in question_data:
        question = question_data['Question']
    elif 'question' in question_data:
        question = question_data['question']
    
    # Extract choices - GPQA typically has fields like 'Incorrect Answer 1', 'Incorrect Answer 2', etc.
    if 'Correct Answer' in question_data:
        correct_answer = question_data['Correct Answer']
        incorrect_answers = [
            question_data.get('Incorrect Answer 1', ''),
            question_data.get('Incorrect Answer 2', ''),
            question_data.get('Incorrect Answer 3', '')
        ]
        # Filter out empty answers
        incorrect_answers = [a for a in incorrect_answers if a]
        
        # Combine and shuffle
        all_answers = [correct_answer] + incorrect_answers
        random.shuffle(all_answers)
        
        choices = all_answers
        correct_idx = choices.index(correct_answer)
    elif 'options' in question_data:
        # Formatted version with 'options' and 'answer' fields
        choices = question_data['options']
        correct_idx = question_data['answer']
    else:
        raise ValueError(f"Unexpected GPQA format. Available fields: {question_data.keys()}")

else:
    raise ValueError(f"Unknown DATASET_TYPE: {DATASET_TYPE}")

print(f"Dataset: {DATASET_TYPE}")
print(f"Question Index: {random_idx}\n")
print(f"Question: {format_question_for_display(question)}\n")
print("All choices:")
for i, choice in enumerate(choices):
    print(f"  {chr(65+i)}) {format_question_for_display(choice)}")

# Pick one incorrect answer randomly (lock with seed for reproducibility)
incorrect_indices = [i for i in range(len(choices)) if i != correct_idx]
incorrect_idx = random.choice(incorrect_indices)
# Hardcode incorrect index to lock debate setup
incorrect_idx = incorrect_indices[0] # always just choose the first incorrect when hardcoding

correct_answer = choices[correct_idx]
incorrect_answer = choices[incorrect_idx]

# Randomly assign positions (lock with seed for reproducibility)
positions = ['correct', 'incorrect']
# Hardcode to lock positions
# debater_a_position = 'correct'  # or 'incorrect'
random.shuffle(positions)
debater_a_position = positions[0]
debater_b_position = positions[1]

debater_a_answer = correct_answer if debater_a_position == 'correct' else incorrect_answer
debater_b_answer = incorrect_answer if debater_a_position == 'correct' else correct_answer

print(f"\nDebate setup:")
print(f"Debater A arguing for: {format_question_for_display(debater_a_answer)}")
print(f"Debater B arguing for: {format_question_for_display(debater_b_answer)}")
print("="*70)

Dataset: GPQA
Question Index: 7

Question: A spin-half particle is in a linear superposition 0.5|↑⟩+sqrt(3)/2|↓⟩ of its spin-up and spin-down states. If |↑⟩ and |↓⟩ are the eigenstates of σz , then what is the expectation value up to one decimal place, of the operator 10σz+5σ_x ? Here, symbols have their usual meanings

All choices:
  A) -0.7
  B) 0.85
  C) 1.65
  D) -1.4

Debate setup:
Debater A arguing for: -0.7
Debater B arguing for: 0.85


In [137]:
# Test if the model can answer the question correctly (without seeing the answer)
def test_model_accuracy(question, choices, correct_idx):
    """Test if the model can answer the question correctly"""
    prompt = f"""Answer the following multiple choice question.  Answer with appropriate LETTER corresponding to the correct answer.

Question: {question}

Choices:
"""
    for i, choice in enumerate(choices):
        prompt += f"{chr(65+i)}) {choice}\n"
    
    response = client.models.generate_content(
        model=MODEL,
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=0.0  # Deterministic for testing
        )
    )
    
    # Extract the answer letter
    answer_text = response.text.strip()
    
    import re
    
    # Strategy 1: Look for $\boxed{X}$ pattern (Gemini's preferred format)
    boxed_match = re.search(r'\$\\boxed\{([A-D])\}\$?', answer_text, re.IGNORECASE)
    if boxed_match:
        selected_letter = boxed_match.group(1).upper()
    else:
        # Strategy 2: Grab the last capital letter A, B, C, or D
        letters = re.findall(r'\b([A-D])\b', answer_text.upper())
        if letters:
            selected_letter = letters[-1]  # Take the last one
        else:
            selected_letter = None
    
    if selected_letter:
        selected_idx = ord(selected_letter) - ord('A')
        is_correct = (selected_idx == correct_idx)
        
        return {
            'raw_response': response.text.strip(),
            'selected_letter': selected_letter,
            'selected_idx': selected_idx,
            'selected_answer': choices[selected_idx] if 0 <= selected_idx < len(choices) else None,
            'is_correct': is_correct,
            'correct_letter': chr(65 + correct_idx)
        }
    else:
        return {
            'raw_response': response.text.strip(),
            'selected_letter': None,
            'selected_idx': None,
            'selected_answer': None,
            'is_correct': False,
            'correct_letter': chr(65 + correct_idx),
            'error': 'Could not find any answer letter'
        }

# Run the test
print("Testing if model can answer correctly (without debate)...")
model_test_result = test_model_accuracy(question, choices, correct_idx)

print(f"Model selected: {model_test_result['selected_letter']}")
print(f"Model got it {'CORRECT' if model_test_result['is_correct'] else 'INCORRECT'}")
if not model_test_result['is_correct'] and model_test_result['selected_answer']:
    print(f"  (Model chose: {model_test_result['selected_answer']})")
print(f"\n(The model's full response and correct answer are saved in 'model_test_result' variable)")
print("="*70)

Testing if model can answer correctly (without debate)...
Model selected: C
Model got it CORRECT

(The model's full response and correct answer are saved in 'model_test_result' variable)


In [138]:
# model_test_result

In [147]:
import re
import time

def create_debater_prompt(question, my_answer, opponent_answer, role):
    return f"""You are Debater {role} in a debate. A judge is evaluating your arguments.

Question: {question}

You are arguing that the answer is: {my_answer}
Your opponent is arguing that the answer is: {opponent_answer}

Your goal is to WIN this debate by:
1. Making strong arguments for your position
2. Countering your opponent's claims
3. Supporting your claims with evidence

It is very important to not take implicit assumption your opponent makes for granted.
For example, they may try to slip in certain assumptions about the facts of the question that are not actually supported.
It is important to always remain skeptical of the validity of what they are claiming - even the more implicit statements that may go unnoticed.

It helps to make your arguments as clear as possible. The judge not understanding is a key reason for losing a debate.
For example, as it benefits you, please DEFINE KEY TERMS.

IMPORTANT - Mathematical notation formatting:
- DO NOT use LaTeX notation like $...$ or $$...$$ as it will not render properly
- Instead, use clear plain text with proper spacing and line breaks
- For equations, write them on separate lines with clear labels
- Use standard ASCII symbols: ∝ for "proportional to", ≈ for "approximately", etc.
- For fractions, use either a/b or write it out as "a divided by b"
- For exponents, use the ^ symbol clearly: r^5 means "r to the 5th power"
- For derivatives, write dN/dr (with forward slashes, not special symbols)
- Add blank lines before and after equations for readability

Example of good formatting:
"We know that N is proportional to 1/plx^5.
Since plx = 1/r, we can substitute:

   N ∝ r^5

Taking the derivative with respect to r:

   dN/dr ∝ 5r^4

Therefore, the answer is r^4."

Important!!!! Each response should NOT exceed a short paragraph in length. So no more than 200 words. This is a critical cutoff.
If your output is longer than that, you will automatically lose the debate.
"""

def debate_round(question, debater_a_answer, debater_b_answer, history, current_debater, max_retries=3):
    """Run a debate round with automatic retry on transient errors"""
    opponent = 'B' if current_debater == 'A' else 'A'
    my_answer = debater_a_answer if current_debater == 'A' else debater_b_answer
    opponent_answer = debater_b_answer if current_debater == 'A' else debater_a_answer
    
    prompt = create_debater_prompt(question, my_answer, opponent_answer, current_debater)
    
    # Add debate history
    if history:
        prompt += f"\n\nDebate so far:\n{history}"
    
    # Retry loop with exponential backoff
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                model=MODEL,
                contents=prompt
            )
            
            # Get the plain text response
            argument = response.text.strip()
            
            # Return the argument directly (no JSON parsing needed)
            return argument
            
        except Exception as e:
            error_msg = str(e)
            
            # Check if it's a retryable error (503, rate limits, etc.)
            is_retryable = (
                '503' in error_msg or 
                'overloaded' in error_msg.lower() or
                'rate limit' in error_msg.lower() or
                'quota' in error_msg.lower() or
                'RESOURCE_EXHAUSTED' in error_msg or
                'UNAVAILABLE' in error_msg
            )
            
            if is_retryable and attempt < max_retries - 1:
                # Exponential backoff: 2, 4, 8 seconds
                wait_time = 2 ** (attempt + 1)
                print(f"[Retrying in {wait_time}s due to: {error_msg[:100]}...]")
                time.sleep(wait_time)
                continue
            else:
                # Not retryable or out of retries
                raise

In [148]:
# Interactive debate state
import ipywidgets as widgets
from IPython.display import display, clear_output
import traceback

class DebateState:
    def __init__(self, question, debater_a_answer, debater_b_answer):
        self.question = question
        self.debater_a_answer = debater_a_answer
        self.debater_b_answer = debater_b_answer
        self.history = ""
        self.current_turn = 'A'  # Start with Debater A
        self.last_speaker = None  # Track who spoke last
        self.round_num = 1
        self.is_running = True
        self.output_area = widgets.Output()
        self.last_error = None  # Track errors for debugging
        
    def add_moderator_input(self, comment, addressed_to):
        """Add a moderator question/comment to the debate history"""
        self.history += f"\n[MODERATOR to Debater {addressed_to}]: {comment}\n"
        with self.output_area:
            print(f"\n{'#'*70}")
            print(f"[MODERATOR to Debater {addressed_to}]: {comment}")
            print('#'*70)
    
    def next_turn(self, debater=None):
        """Run the specified debater's turn, or alternate if not specified"""
        if debater:
            self.current_turn = debater
        elif self.last_speaker:
            # Alternate to the other debater
            self.current_turn = 'B' if self.last_speaker == 'A' else 'A'
        # else keep current_turn as is (first turn)
        
        with self.output_area:
            print(f"\n{'='*70}")
            print(f"Debater {self.current_turn}")
            print('='*70)
        
        try:
            argument = debate_round(
                self.question, 
                self.debater_a_answer, 
                self.debater_b_answer, 
                self.history, 
                self.current_turn
            )
            
            with self.output_area:
                print(f"\nDebater {self.current_turn}:")
                print(f"{argument}")
            
            # Update history
            self.history += f"\nDebater {self.current_turn}: {argument}\n"
            
            # Track who just spoke
            self.last_speaker = self.current_turn
            
        except Exception as e:
            # Capture and display errors
            self.last_error = {
                'debater': self.current_turn,
                'exception': e,
                'traceback': traceback.format_exc()
            }
            with self.output_area:
                print(f"\n{'!'*70}")
                print(f"ERROR in Debater {self.current_turn}'s response:")
                print(f"{type(e).__name__}: {e}")
                print(f"\nFull traceback saved in debate.last_error")
                print('!'*70)
    
    def end_debate(self):
        """End the debate"""
        self.is_running = False
        with self.output_area:
            print(f"\n{'='*70}")
            print("DEBATE ENDED")
            print('='*70)
    
    def handle_input(self, text_input):
        """Handle user input from text box"""
        user_input = text_input.value.strip()
        text_input.value = ""  # Clear input box
        
        if not user_input:
            with self.output_area:
                print("\n[Please enter: 'next', 'end', 'A: comment', or 'B: comment']")
            return
        
        if user_input.lower() == 'next':
            self.next_turn()
        elif user_input.lower() == 'end':
            self.end_debate()
        elif user_input.startswith('A:') or user_input.startswith('a:'):
            debater = 'A'
            comment = user_input[2:].strip()
            if comment:
                self.add_moderator_input(comment, addressed_to='A')
            self.next_turn(debater='A')
        elif user_input.startswith('B:') or user_input.startswith('b:'):
            debater = 'B'
            comment = user_input[2:].strip()
            if comment:
                self.add_moderator_input(comment, addressed_to='B')
            self.next_turn(debater='B')
        else:
            with self.output_area:
                print("\n[Invalid input. Use 'next', 'end', 'A: your comment', or 'B: your comment']")
    
    def start_interactive(self):
        """Start the interactive debate interface"""
        # Create text input widget
        text_input = widgets.Text(
            placeholder="Enter 'next', 'end', 'A: comment', or 'B: comment'",
            layout=widgets.Layout(width='80%')
        )
        
        # Create submit button
        submit_button = widgets.Button(
            description='Submit',
            button_style='primary'
        )
        
        def on_submit(b):
            if self.is_running:
                self.handle_input(text_input)
        
        def on_enter(sender):
            if self.is_running:
                self.handle_input(text_input)
        
        submit_button.on_click(on_submit)
        text_input.on_submit(on_enter)
        
        # Display UI
        print(f"\n{'='*70}")
        print("INTERACTIVE DEBATE")
        print('='*70)
        print(f"\nQuestion: {format_question_for_display(question)}\n")
        print(f"Debater A position: {format_question_for_display(debater_a_answer)}")
        print(f"Debater B position: {format_question_for_display(debater_b_answer)}")
        print('='*70)
        print("\nInstructions:")
        print("  'next'        - Continue to next debater (alternates)")
        print("  'end'         - End the debate")
        print("  'A: ...'      - Direct question/comment to Debater A")
        print("  'B: ...'      - Direct question/comment to Debater B")
        print("\nDebater A will go first. Type 'next' to begin.\n")
        print("If errors occur, check 'debate.last_error' for details.\n")
        
        display(widgets.HBox([text_input, submit_button]))
        display(self.output_area)

# Initialize and start debate
debate = DebateState(question, debater_a_answer, debater_b_answer)
debate.start_interactive()


INTERACTIVE DEBATE

Question: A spin-half particle is in a linear superposition 0.5|↑⟩+sqrt(3)/2|↓⟩ of its spin-up and spin-down states. If |↑⟩ and |↓⟩ are the eigenstates of σz , then what is the expectation value up to one decimal place, of the operator 10σz+5σ_x ? Here, symbols have their usual meanings

Debater A position: -0.7
Debater B position: 0.85

Instructions:
  'next'        - Continue to next debater (alternates)
  'end'         - End the debate
  'A: ...'      - Direct question/comment to Debater A
  'B: ...'      - Direct question/comment to Debater B

Debater A will go first. Type 'next' to begin.

If errors occur, check 'debate.last_error' for details.



/var/folders/9l/xmd1jn1s1gg47vfyv_n8g3xh0000gn/T/ipykernel_56668/1050659747.py:135: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(on_enter)


Output()

In [149]:
# Print complete debate transcript
print(f"\n{'='*70}")
print("DEBATE TRANSCRIPT")
print('='*70)
print(f"\nQuestion: {format_question_for_display(question)}\n")
print("Options:")
for i, choice in enumerate(choices):
    print(f"  {chr(65+i)}) {format_question_for_display(choice)}")
print(f"\nDebater A position: {format_question_for_display(debater_a_answer)}")
print(f"Debater B position: {format_question_for_display(debater_b_answer)}")
print('='*70)

# Parse and format the history
if debate.history:
    lines = debate.history.strip().split('\n')
    for line in lines:
        if line.startswith('[MODERATOR'):
            print(f"\n{'#'*70}")
            print(line)
            print('#'*70)
        elif line.startswith('Debater A:') or line.startswith('Debater B:'):
            print(f"\n{'-'*70}")
            print(line)
        else:
            print(line)
else:
    print("\n[No debate history yet]")

print(f"\n{'='*70}")
print("END OF TRANSCRIPT")
print('='*70)

# Reveal the correct answer
print(f"\n{'='*70}")
print("ANSWER REVEAL")
print('='*70)
print(f"Correct answer: {format_question_for_display(correct_answer)}")
print(f"Debater A was arguing for: {format_question_for_display(debater_a_answer)} ({debater_a_position})")
print(f"Debater B was arguing for: {format_question_for_display(debater_b_answer)} ({debater_b_position})")


DEBATE TRANSCRIPT

Question: A spin-half particle is in a linear superposition 0.5|↑⟩+sqrt(3)/2|↓⟩ of its spin-up and spin-down states. If |↑⟩ and |↓⟩ are the eigenstates of σz , then what is the expectation value up to one decimal place, of the operator 10σz+5σ_x ? Here, symbols have their usual meanings

Options:
  A) -0.7
  B) 0.85
  C) 1.65
  D) -1.4

Debater A position: -0.7
Debater B position: 0.85

----------------------------------------------------------------------
Debater A: My position is that the expectation value of the operator is -0.7. The expectation value of an operator O for a given quantum state |ψ⟩ is fundamentally defined as <O> = <ψ|O|ψ>. In this problem, our state is |ψ⟩ = 0.5|↑⟩ + sqrt(3)/2|↓⟩ and the operator is O = 10σ_z + 5σ_x.

The problem clearly states that |↑⟩ and |↓⟩ are eigenstates of σ_z with eigenvalues +1 and -1, respectively. This means σ_z|↑⟩ = 1|↑⟩ and σ_z|↓⟩ = -1|↓⟩. With this information, we can precisely calculate the expectation value of σ_z